### Spark notebook ###

This notebook will only work in a Jupyter session running on `mathmadslinux2p`.

You can start your own Jupyter session on `mathmadslinux2p` and open this notebook in Chrome on the MADS Windows server by

**Steps**

1. Login to the MADS Windows server using https://mathportal.canterbury.ac.nz/.
2. Download or copy this notebook to your home directory.
3. Open powershell and run `ssh mathmadslinux2p`.
4. Run `start_pyspark_notebook` or `/opt/anaconda3/bin/jupyter-notebook --ip 132.181.129.68 --port $((8000 + $((RANDOM % 999))))`.
5. Copy / paste the url provided in the shell window into Chrome on the MADS Windows server.
6. Open the notebook from the Jupyter root directory (which is your home directory).
7. Run `start_spark()` to start a spark session in the notebook.
8. Run `stop_spark()` before closing the notebook or kill your spark application by hand using the link in the Spark UI.

In [1]:
# Run this cell to import pyspark and to define start_spark() and stop_spark()

import findspark

findspark.init()

import getpass
import pandas as pd
import pyspark
import random
import re

from IPython.display import display, HTML
from pyspark import SparkContext
from pyspark.sql import SparkSession


# Functions used below

def username():
    """Get username with any domain information removed.
    """

    return re.sub('@.*', '', getpass.getuser())


def dict_to_html(d):
    """Convert a Python dictionary into a two column table for display.
    """

    html = []

    html.append(f'<table width="100%" style="width:100%; font-family: monospace;">')
    for k, v in d.items():
        html.append(f'<tr><td style="text-align:left;">{k}</td><td>{v}</td></tr>')
    html.append(f'</table>')

    return ''.join(html)


def show_as_html(df, n=10):
    """Leverage existing pandas jupyter integration to show a spark dataframe as html.
    
    Args:
        n (int): number of rows to show (default: 20)
    """

    display(df.limit(n).toPandas())

    
def display_spark():
    """Display the status of the active Spark session if one is currently running.
    """
    
    if 'spark' in globals() and 'sc' in globals():

        name = sc.getConf().get("spark.app.name")
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:green">active</span></b>, look for <code>{name}</code> under the running applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'<li><a href="{sc.uiWebUrl}" target="_blank">Spark Application UI</a></li>',
            f'</ul>',
            f'<p><b>Config</b></p>',
            dict_to_html(dict(sc.getConf().getAll())),
            f'<p><b>Notes</b></p>',
            f'<ul>',
            f'<li>The spark session <code>spark</code> and spark context <code>sc</code> global variables have been defined by <code>start_spark()</code>.</li>',
            f'<li>Please run <code>stop_spark()</code> before closing the notebook or restarting the kernel or kill <code>{name}</code> by hand using the link in the Spark UI.</li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))
        
    else:
        
        html = [
            f'<p><b>Spark</b></p>',
            f'<p>The spark session is <b><span style="color:red">stopped</span></b>, confirm that <code>{username() + " (jupyter)"}</code> is under the completed applications section in the Spark UI.</p>',
            f'<ul>',
            f'<li><a href="http://mathmadslinux2p.canterbury.ac.nz:8080/" target="_blank">Spark UI</a></li>',
            f'</ul>',
        ]
        display(HTML(''.join(html)))


# Functions to start and stop spark

def start_spark(executor_instances=2, executor_cores=1, worker_memory=1, master_memory=1):
    """Start a new Spark session and define globals for SparkSession (spark) and SparkContext (sc).
    
    Args:
        executor_instances (int): number of executors (default: 2)
        executor_cores (int): number of cores per executor (default: 1)
        worker_memory (float): worker memory (default: 1)
        master_memory (float): master memory (default: 1)
    """

    global spark
    global sc

    user = username()
    
    cores = executor_instances * executor_cores
    partitions = cores * 4
    port = 4000 + random.randint(1, 999)

    spark = (
        SparkSession.builder
        .master("spark://masternode2:7077")
        .config("spark.driver.extraJavaOptions", f"-Dderby.system.home=/tmp/{user}/spark/")
        .config("spark.dynamicAllocation.enabled", "false")
        .config("spark.executor.instances", str(executor_instances))
        .config("spark.executor.cores", str(executor_cores))
        .config("spark.cores.max", str(cores))
        .config("spark.executor.memory", f"{worker_memory}g")
        .config("spark.driver.memory", f"{master_memory}g")
        .config("spark.driver.maxResultSize", "0")
        .config("spark.sql.shuffle.partitions", str(partitions))
        .config("spark.ui.port", str(port))
        .appName(user + " (jupyter)")
        .getOrCreate()
    )
    sc = SparkContext.getOrCreate()
    
    display_spark()

    
def stop_spark():
    """Stop the active Spark session and delete globals for SparkSession (spark) and SparkContext (sc).
    """

    global spark
    global sc

    if 'spark' in globals() and 'sc' in globals():

        spark.stop()

        del spark
        del sc

    display_spark()


# Make css changes to improve spark output readability

html = [
    '<style>',
    'pre { white-space: pre !important; }',
    'table.dataframe td { white-space: nowrap !important; }',
    'table.dataframe thead th:first-child, table.dataframe tbody th { display: none; }',
    '</style>',
]
display(HTML(''.join(html)))

In [31]:
# Run this cell to start a spark session in this notebook

start_spark(executor_instances=4, executor_cores=2, worker_memory=4, master_memory=4)

spark.app.name,kda115 (jupyter)
spark.dynamicAllocation.enabled,false
spark.executor.instances,4
spark.driver.memory,4g
spark.executor.memory,4g
spark.master,spark://masternode2:7077
spark.executor.id,driver
spark.executor.cores,2
spark.app.startTime,1726969158275
spark.driver.host,mathmadslinux2p.canterbury.ac.nz
spark.driver.port,44141


## Processing Question 1

In [83]:
# Write your imports and code here or insert cells below

from pyspark.sql import Row, DataFrame, Window, functions as F
from pyspark.sql.types import *

### A. Give an overview of the structure of the datasets, including their sizes, formats, data types, and how each dataset has been stored in HDFS.


### 1. List the files in the HDFS directory to understand the dataset's structure

In [3]:
# Listing the contents of the HDFS directory
! hdfs dfs -ls hdfs:///data/msd/

Found 4 items
drwxr-xr-x   - jsw93 supergroup          0 2021-09-29 10:35 hdfs:///data/msd/audio
drwxr-xr-x   - jsw93 supergroup          0 2021-09-29 10:35 hdfs:///data/msd/genre
drwxr-xr-x   - jsw93 supergroup          0 2021-09-29 10:28 hdfs:///data/msd/main
drwxr-xr-x   - jsw93 supergroup          0 2021-09-29 10:35 hdfs:///data/msd/tasteprofile


### 2. Get the sizes of each dataset

#### 1. Get the overall size of each directory

In [5]:
# Get the size of each file in the dataset
! hdfs dfs -du -h hdfs:///data/msd/

12.3 G   98.1 G   hdfs:///data/msd/audio
30.1 M   241.0 M  hdfs:///data/msd/genre
174.4 M  1.4 G    hdfs:///data/msd/main
490.4 M  3.8 G    hdfs:///data/msd/tasteprofile


In [1]:
# using -du and -s provide a summary of disk usage for the entire directory
! hdfs dfs -du -s -h hdfs:///data/msd/

12.9 G  103.5 G  hdfs:///data/msd


#### 2.  Explore file size of  Audio directory

In [20]:
# Displays file sizes and directory of Audio dataset 
! hdfs dfs -du -s -h /data/msd/audio
# Get the size of audio file in the dataset
! hdfs dfs -du -h /data/msd/audio/

12.3 G  98.1 G  /data/msd/audio
103.0 K  824.3 K  /data/msd/audio/attributes
12.2 G   97.8 G   /data/msd/audio/features
40.3 M   322.1 M  /data/msd/audio/statistics


In [10]:
# File size and directory contained in attributes subdirectory
! hdfs dfs -du -s -h /data/msd/audio/attributes

# Displays sizes attributes subdirectory

! hdfs dfs -du -h /data/msd/audio/attributes

103.0 K  824.3 K  /data/msd/audio/attributes
1.0 K   8.2 K    /data/msd/audio/attributes/msd-jmir-area-of-moments-all-v1.0.attributes.csv
671     5.2 K    /data/msd/audio/attributes/msd-jmir-lpc-all-v1.0.attributes.csv
484     3.8 K    /data/msd/audio/attributes/msd-jmir-methods-of-moments-all-v1.0.attributes.csv
898     7.0 K    /data/msd/audio/attributes/msd-jmir-mfcc-all-v1.0.attributes.csv
777     6.1 K    /data/msd/audio/attributes/msd-jmir-spectral-all-all-v1.0.attributes.csv
777     6.1 K    /data/msd/audio/attributes/msd-jmir-spectral-derivatives-all-all-v1.0.attributes.csv
12.0 K  96.2 K   /data/msd/audio/attributes/msd-marsyas-timbral-v1.0.attributes.csv
9.8 K   78.0 K   /data/msd/audio/attributes/msd-mvd-v1.0.attributes.csv
1.4 K   10.9 K   /data/msd/audio/attributes/msd-rh-v1.0.attributes.csv
34.1 K  272.8 K  /data/msd/audio/attributes/msd-rp-v1.0.attributes.csv
3.8 K   30.8 K   /data/msd/audio/attributes/msd-ssd-v1.0.attributes.csv
9.8 K   78.0 K   /data/msd/audio/attribut

In [11]:
# File size and directory contained in features subdirectory
! hdfs dfs -du -s -h /data/msd/audio/features

# Displays sizes features subdirectory

! hdfs dfs -du -h /data/msd/audio/features

12.2 G  97.8 G  /data/msd/audio/features
65.5 M   524.2 M  /data/msd/audio/features/msd-jmir-area-of-moments-all-v1.0.csv
53.1 M   424.6 M  /data/msd/audio/features/msd-jmir-lpc-all-v1.0.csv
35.8 M   286.5 M  /data/msd/audio/features/msd-jmir-methods-of-moments-all-v1.0.csv
70.8 M   566.1 M  /data/msd/audio/features/msd-jmir-mfcc-all-v1.0.csv
51.1 M   408.9 M  /data/msd/audio/features/msd-jmir-spectral-all-all-v1.0.csv
51.1 M   408.9 M  /data/msd/audio/features/msd-jmir-spectral-derivatives-all-all-v1.0.csv
412.2 M  3.2 G    /data/msd/audio/features/msd-marsyas-timbral-v1.0.csv
1.3 G    10.3 G   /data/msd/audio/features/msd-mvd-v1.0.csv
240.3 M  1.9 G    /data/msd/audio/features/msd-rh-v1.0.csv
4.0 G    32.3 G   /data/msd/audio/features/msd-rp-v1.0.csv
640.6 M  5.0 G    /data/msd/audio/features/msd-ssd-v1.0.csv
1.4 G    11.5 G   /data/msd/audio/features/msd-trh-v1.0.csv
3.9 G    31.0 G   /data/msd/audio/features/msd-tssd-v1.0.csv


In [12]:
# File size and directory contained in statistics subdirectory
! hdfs dfs -du -s -h /data/msd/audio/statistics

# Displays sizes statistics subdirectory

! hdfs dfs -du -h /data/msd/audio/statistics

40.3 M  322.1 M  /data/msd/audio/statistics
40.3 M  322.1 M  /data/msd/audio/statistics/sample_properties.csv.gz


In [16]:
# Directory trees of Audio dataset
! hdfs dfs -ls -R /data/msd/audio | awk '{print $8}' | sed -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/'

 |-----attributes
 |-------msd-jmir-area-of-moments-all-v1.0.attributes.csv
 |-------msd-jmir-lpc-all-v1.0.attributes.csv
 |-------msd-jmir-methods-of-moments-all-v1.0.attributes.csv
 |-------msd-jmir-mfcc-all-v1.0.attributes.csv
 |-------msd-jmir-spectral-all-all-v1.0.attributes.csv
 |-------msd-jmir-spectral-derivatives-all-all-v1.0.attributes.csv
 |-------msd-marsyas-timbral-v1.0.attributes.csv
 |-------msd-mvd-v1.0.attributes.csv
 |-------msd-rh-v1.0.attributes.csv
 |-------msd-rp-v1.0.attributes.csv
 |-------msd-ssd-v1.0.attributes.csv
 |-------msd-trh-v1.0.attributes.csv
 |-------msd-tssd-v1.0.attributes.csv
 |-----features
 |-------msd-jmir-area-of-moments-all-v1.0.csv
 |---------part-00000.csv.gz
 |---------part-00001.csv.gz
 |---------part-00002.csv.gz
 |---------part-00003.csv.gz
 |---------part-00004.csv.gz
 |---------part-00005.csv.gz
 |---------part-00006.csv.gz
 |---------part-00007.csv.gz
 |-------msd-jmir-lpc-all-v1.0.csv
 |---------part-00000.csv.gz
 |---------part-000

#### 3. Explore file size genre directory

In [18]:
# Get the size of genre file in the dataset
! hdfs dfs -du -s -h /data/msd/genre

# Get the file seze of genre subdirectory 
! hdfs dfs -du -h /data/msd/genre/

30.1 M  241.0 M  /data/msd/genre
11.1 M  88.7 M  /data/msd/genre/msd-MAGD-genreAssignment.tsv
8.4 M   67.3 M  /data/msd/genre/msd-MASD-styleAssignment.tsv
10.6 M  85.0 M  /data/msd/genre/msd-topMAGD-genreAssignment.tsv


In [19]:
# Directory trees of Genre directory
! hdfs dfs -ls -R /data/msd/genre | awk '{print $8}' | sed -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/' 

 |-----msd-MAGD-genreAssignment.tsv
 |-----msd-MASD-styleAssignment.tsv
 |-----msd-topMAGD-genreAssignment.tsv


#### 4. Explore file size of Main directory

In [77]:
# Get the size of Main file in the dataset
! hdfs dfs -du -s -h /data/msd/main

# Get the file seze of main subdirectory 
! hdfs dfs -du -h /data/msd/main/summary

174.4 M  1.4 G  /data/msd/main
55.9 M   447.5 M  /data/msd/main/summary/analysis.csv.gz
118.5 M  947.7 M  /data/msd/main/summary/metadata.csv.gz


In [22]:
# Directory trees of Main directory
! hdfs dfs -ls -R /data/msd/main | awk '{print $8}' | sed -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/' 

 |-----summary
 |-------analysis.csv.gz
 |-------metadata.csv.gz


#### 5. Explore file size of Tasteprofile directory

In [24]:
# Get the size of Tasteprofile file in the directory
! hdfs dfs -du -s -h /data/msd/tasteprofile

# Get the file size of Tasteprofile subdirectory 
! hdfs dfs -du -h /data/msd/tasteprofile

490.4 M  3.8 G  /data/msd/tasteprofile
2.0 M    16.2 M  /data/msd/tasteprofile/mismatches
488.4 M  3.8 G   /data/msd/tasteprofile/triplets.tsv


In [26]:
# Get the size of mismatches file in the Tasteprofile directory
! hdfs dfs -du -s -h /data/msd/tasteprofile/mismatches

# Get the file size of file contain in mismatches subdirectory 
! hdfs dfs -du -h /data/msd/tasteprofile/mismatches

2.0 M  16.2 M  /data/msd/tasteprofile/mismatches
89.2 K  713.6 K  /data/msd/tasteprofile/mismatches/sid_matches_manually_accepted.txt
1.9 M   15.5 M   /data/msd/tasteprofile/mismatches/sid_mismatches.txt


In [28]:
# Directory trees of Tasteprofile directory
! hdfs dfs -ls -R /data/msd/tasteprofile | awk '{print $8}' | sed -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/'

 |-----mismatches
 |-------sid_matches_manually_accepted.txt
 |-------sid_mismatches.txt
 |-----triplets.tsv
 |-------part-00000.tsv.gz
 |-------part-00001.tsv.gz
 |-------part-00002.tsv.gz
 |-------part-00003.tsv.gz
 |-------part-00004.tsv.gz
 |-------part-00005.tsv.gz
 |-------part-00006.tsv.gz
 |-------part-00007.tsv.gz


In [29]:
# The structure of all Million Songs Dataset (MSD)
! hdfs dfs -ls -R /data/msd/ | awk '{print $8}' | sed -e 's/[^-][^\/]*\//--/g' -e 's/^/ /' -e 's/-/|/'

 |---audio
 |-----attributes
 |-------msd-jmir-area-of-moments-all-v1.0.attributes.csv
 |-------msd-jmir-lpc-all-v1.0.attributes.csv
 |-------msd-jmir-methods-of-moments-all-v1.0.attributes.csv
 |-------msd-jmir-mfcc-all-v1.0.attributes.csv
 |-------msd-jmir-spectral-all-all-v1.0.attributes.csv
 |-------msd-jmir-spectral-derivatives-all-all-v1.0.attributes.csv
 |-------msd-marsyas-timbral-v1.0.attributes.csv
 |-------msd-mvd-v1.0.attributes.csv
 |-------msd-rh-v1.0.attributes.csv
 |-------msd-rp-v1.0.attributes.csv
 |-------msd-ssd-v1.0.attributes.csv
 |-------msd-trh-v1.0.attributes.csv
 |-------msd-tssd-v1.0.attributes.csv
 |-----features
 |-------msd-jmir-area-of-moments-all-v1.0.csv
 |---------part-00000.csv.gz
 |---------part-00001.csv.gz
 |---------part-00002.csv.gz
 |---------part-00003.csv.gz
 |---------part-00004.csv.gz
 |---------part-00005.csv.gz
 |---------part-00006.csv.gz
 |---------part-00007.csv.gz
 |-------msd-jmir-lpc-all-v1.0.csv
 |---------part-00000.csv.gz
 |------

### B. Figure out how to load each of the different types of datasets, even if you infer schema and only load a small subset of the data for testing.

#### 1. Loading the  Attributes Dataset

In [1]:
# Load the small sample attribute feature dataset using HDFS command
! hdfs dfs -cat /data/msd/audio/attributes/msd-jmir-area-of-moments-all-v1.0.attributes.csv | head

Area_Method_of_Moments_Overall_Standard_Deviation_1,real
Area_Method_of_Moments_Overall_Standard_Deviation_2,real
Area_Method_of_Moments_Overall_Standard_Deviation_3,real
Area_Method_of_Moments_Overall_Standard_Deviation_4,real
Area_Method_of_Moments_Overall_Standard_Deviation_5,real
Area_Method_of_Moments_Overall_Standard_Deviation_6,real
Area_Method_of_Moments_Overall_Standard_Deviation_7,real
Area_Method_of_Moments_Overall_Standard_Deviation_8,real
Area_Method_of_Moments_Overall_Standard_Deviation_9,real
Area_Method_of_Moments_Overall_Standard_Deviation_10,real


#### 2. Loading the  Features Dataset

In [2]:
# Load the small sample feature dataset using HDFS command
! hdfs dfs -cat /data/msd/audio/features/msd-jmir-area-of-moments-all-v1.0.csv/part-00000.csv.gz | gunzip | head

1.431,6713.0,52600.0,160600000.0,1264000000.0,9943000000.0,7.086e+12,11400000000.0,89730000000.0,3.465e+15,5.252,11580.0,90080.0,-179100000.0,-1396000000.0,-10870000000.0,6.236e+12,12580000000.0,98020000000.0,2.97e+15,'TRMMMYQ128F932D901'
0.9864,3361.0,24270.0,40110000.0,287800000.0,2064000000.0,8.837e+11,2596000000.0,18630000000.0,3.232e+14,2.773,5774.0,41490.0,-44600000.0,-320900000.0,-2307000000.0,7.756e+11,2885000000.0,20760000000.0,2.883e+14,'TRMMMKD128F425225D'
1.791,6717.0,57790.0,160900000.0,1385000000.0,11910000000.0,7.105e+12,12520000000.0,1.077e+11,4.52e+15,6.43,11600.0,99690.0,-179500000.0,-1544000000.0,-13270000000.0,6.255e+12,13950000000.0,1.2e+11,3.976e+15,'TRMMMRX128F93187D9'
2.209,3371.0,34750.0,40350000.0,412300000.0,4210000000.0,8.912e+11,3710000000.0,37900000000.0,9.415e+14,5.734,5792.0,58320.0,-44870000.0,-454600000.0,-4603000000.0,7.828e+11,4083000000.0,41370000000.0,8.199e+14,'TRMMMCH128F425532C'
0.6846,6708.0,30690.0,160400000.0,748900000.0,3492000000.0,7.073e+1

#### 3. Loading the Statistic Dataset

In [3]:
# Load the small sample attribute statistic dataset using HDFS command
! hdfs dfs -cat /data/msd/audio/statistics/sample_properties.csv.gz | gunzip | head

track_id,title,artist_name,duration,7digita_Id,sample_bitrate,sample_length,sample_rate,sample_mode,sample_version,filesize
TRMMMYQ128F932D901,"Silent Night","Faster Pussy cat",252.05506,7032331,128,60.1935770567,22050,1,2,960887
TRMMMKD128F425225D,"Tanssi vaan",Karkkiautomaatti,156.55138,1514808,64,30.2244270016,22050,1,2,242038
TRMMMRX128F93187D9,"No One Could Ever","Hudson Mohawke",138.97098,6945353,128,60.1935770567,22050,1,2,960887
TRMMMCH128F425532C,"Si Vos Querés","Yerba Brava",145.05751,2168257,64,30.2083516484,22050,1,2,240534
TRMMMWA128F426B589,"Tangle Of Aspens","Der Mystic",514.29832,2264873,64,60.3382103611,22050,1,2,480443
TRMMMXN128F42936A5,"Symphony No. 1 G minor ""Sinfonie Serieuse""/Allegro con energia","David Montgomery",816.53506,3360982,128,30.1360348456,44100,0,1,481070
TRMMMLR128F1494097,"We Have Got Love","Sasha / Turbulence",212.37506,552626,64,60.3542857143,22050,1,2,480686
TRMMMBB12903CB7D21,"2 Da Beat Ch'yall","Kris Kross",221.20444,6435649,128,30.1360348456

#### 4. Loading the Genre Dataset

In [3]:
# Load the small sample genre dataset using HDFS command
! hdfs dfs -cat /data/msd/genre/msd-MAGD-genreAssignment.tsv | head

TRAAAAK128F9318786	Pop_Rock
TRAAAAV128F421A322	Pop_Rock
TRAAAAW128F429D538	Rap
TRAAABD128F429CF47	Pop_Rock
TRAAACV128F423E09E	Pop_Rock
TRAAADT12903CCC339	Easy_Listening
TRAAAED128E0783FAB	Vocal
TRAAAEF128F4273421	Pop_Rock
TRAAAEM128F93347B9	Electronic
TRAAAFD128F92F423A	Pop_Rock
cat: Unable to write to output stream.


 #### 5. Loading the Main Dataset

In [4]:
# Load the small sample main dataset using HDFS command
! hdfs dfs -cat /data/msd/main/summary/metadata.csv.gz | gunzip | head

analyzer_version,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,genre,idx_artist_terms,idx_similar_artists,release,release_7digitalid,song_hotttnesss,song_id,title,track_7digitalid
,4069,0.6498221002008776,0.3940318927141434,ARYZTJS1187B98C555,,,,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,44895,,0,0,Monster Ballads X-Mas,633681,0.5428987432910862,SOQMMHC12AB0180CB8,Silent Night,7032331
,113480,0.4396039666767154,0.3569921077564064,ARMVN3U1187FB3A1EB,,,,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,-1,,0,0,Karkuteillä,145266,0.2998774882739778,SOVFVAK12A8C1350D9,Tanssi vaan,1514808
,63531,0.6436805720579895,0.4375038365946544,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,-1,,0,0,Butter,625706,0.6178709693948196,SOGTUKN12AB017F4F1,No One Could Ever,6945353
,65051,0.44850115965646636,0.37234906851712235,

#### 6. Loading the Taste Profile Dataset

In [5]:
# Load the small sample TasteProfile triplets dataset using HDFS command
! hdfs dfs -cat /data/msd/tasteprofile/triplets.tsv/part-00000.tsv.gz | gunzip | head

b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOAKIMP12A8C130995	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOAPDEY12A81C210A9	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBBMDR12A8C13253B	2
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBFNSP12AF72A0E22	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBFOVM12A58A7D494	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBNZDC12A6D4FC103	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBSUJE12A6D4F8CF5	2
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBVFZR12A6D4F8AE3	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBXALG12A8C13C108	1
b80344d063b5ccb3212f76538f3d9e43d87dca9e	SOBXHDL12A81C204C0	1

gzip: stdout: Broken pipe
cat: Unable to write to output stream.


In [4]:
# Load the small sample TasteProfile missmatches dataset using HDFS command
! hdfs dfs -cat /data/msd/tasteprofile/mismatches/sid_matches_manually_accepted.txt | head

9d8
< ERROR: <SOFQHZM12A8C142342 TRMWMFG128F92FFEF2> Josipa Lisac  -  razloga  !=  Lisac Josipa  -  1000 razloga
19d17
< ERROR: <SODXUTF12AB018A3DA TRMWPCD12903CCE5ED> Lutan Fyah  -  Nuh Matter the Crisis Feat. Midnite  !=  Midnite  -  Nah Matter the Crisis
29d26
< ERROR: <SOASCRF12A8C1372E6 TRMHIPJ128F426A2E2> Gaetano Donizetti  -  L'Elisir d'Amore: Act Two: Come sen va contento!  !=  Gianandrea Gavazzeni_ Orchestra E Coro Del Maggio Musicale Fiorentino_ Carlo Bergonzi_ Renata Scotto  -  L'Elisir D'Amore_ Act 2: Come Sen Va Contento (Adina) (Donizetti)
33d29
< ERROR: <SOITDUN12A58A7AACA TRMHXGK128F42446AB> C.J. Chenier  -  Ay, Ai Ai  !=  Clifton Chenier  -  Ay_ Ai Ai
52d47
< ERROR: <SOLZXUM12AB018BE39 TRMRSOF12903CCF516> 許志安  -  男人最痛  !=  Andy Hui  -  Nan Ren Zui Tong
cat: Unable to write to output stream.


### C. Count the number of rows in each of the datasets. How do the counts compare to the total number of unique songs?

#### 1. Counting Rows for the  Attributes Directory

In [2]:
# Count the rows for each of file in the attributes and save as csv
!for i in `hdfs dfs -ls -R /data/msd/audio/attributes | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | wc -l; done > attributes_row_num.csv

In [2]:
%%bash
# Define the HDFS path for the attributes directory
attributes_dir="/data/msd/audio/attributes"

# Initialize a variable to store the total row count
total_rows=0

# Loop through each file in the attributes directory and count the rows (including header)
for file in $(hdfs dfs -ls $attributes_dir | awk '{print $8}')
do
  # Get the number of rows in the current file
  row_count=$(hdfs dfs -cat $file | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
done

# Print the total number of rows
echo "Total Row Count for Attribute Directory: $total_rows"

Total number of rows across all files: 3929


#### 2. Counting Rows for the Feature Directory

In [4]:
# Count the rows for each of file in the features and save as csv
!for i in `hdfs dfs -ls -R /data/msd/audio/features | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | zcat | wc -l; done > features_count.csv

cat: `/data/msd/audio/features/msd-jmir-area-of-moments-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-jmir-lpc-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-jmir-methods-of-moments-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-jmir-mfcc-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-jmir-spectral-all-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-jmir-spectral-derivatives-all-all-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-marsyas-timbral-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-mvd-v1.0.csv': Is a directory

gzip: stdin: unexpected end of file
cat: `/data/msd/audio/features/msd-rh-v1.0.csv': Is a direct

In [12]:
%%bash
# Define the HDFS path for the features directory
features_dir="/data/msd/audio/features"

# Initialize a variable to store the total row count
total_rows=0

# Loop through each part file in the features directory and count the rows (including header)
for file in $(hdfs dfs -ls -R $features_dir | grep .gz | awk '{print $8}')
do
  # Get the number of rows in the current compressed part file, and suppress error messages
  row_count=$(hdfs dfs -cat $file 2>/dev/null | gunzip | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
done

# Print the total number of rows
echo "Total Row Count for all Feature Subdirectories: $total_rows"

Total Row Count for all Feature Subdirectories: 12927867


#### 3. Counting Rows for the Genre Directory

In [5]:
# Count the rows for each of file in the genre and save as csv
!for i in `hdfs dfs -ls -R /data/msd/genre | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | wc -l; done > genre_count.csv

In [4]:
%%bash
# Define the HDFS path for the genre directory
genre_dir="/data/msd/genre"

# Initialize a variable to store the total row count
total_rows=0

# Loop through each .tsv file in the genre directory and count the rows (including header)
for file in $(hdfs dfs -ls $genre_dir | grep .tsv | awk '{print $8}')
do
  # Get the number of rows in the current file
  row_count=$(hdfs dfs -cat $file | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
  
  # Print the row count for each file (optional)
  echo "File: $file, Rows: $row_count"
done

# Print the total number of rows
echo "Total Row Count for Genre Directory: $total_rows"

File: /data/msd/genre/msd-MAGD-genreAssignment.tsv, Rows: 422714
File: /data/msd/genre/msd-MASD-styleAssignment.tsv, Rows: 273936
File: /data/msd/genre/msd-topMAGD-genreAssignment.tsv, Rows: 406427
Total Row Count for Genre Directory: 1103077


#### 4. Counting Rows for the Statistic Directory

In [6]:
# Count the rows for each of file in the statistic and save as csv
!for i in `hdfs dfs -ls -R /data/msd/audio/statistics | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | zcat | wc -l; done > statistics_count.csv

In [11]:
%%bash
# Define the HDFS path for the statistics file
statistics_file="hdfs:///data/msd/audio/statistics/sample_properties.csv.gz"

# Count the number of rows (including header) in the compressed file, and redirect errors to /dev/null
row_count=$(hdfs dfs -cat $statistics_file 2>/dev/null | gunzip | wc -l)

# Print the total number of rows
echo "Total Row Count for Statistics Subdirectories: $row_count"

Total Row Count for Statistics Subdirectories: 992866


#### 5. Counting Rows for the Main Directory

In [7]:
# Count the rows for each of file in the statistic and save as csv
!for i in `hdfs dfs -ls -R /data/msd/main/summary | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | zcat | wc -l; done > main_count.csv

In [7]:
%%bash
# Define the HDFS path for the summary directory in main
summary_dir="/data/msd/main/summary"

# Initialize a variable to store the total row count
total_rows=0

# Loop through each .csv.gz file in the summary directory and count the rows (including header)
for file in $(hdfs dfs -ls $summary_dir | grep .gz | awk '{print $8}')
do
  # Get the number of rows in the current compressed file
  row_count=$(hdfs dfs -cat $file | gunzip | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
  
  # Print the row count for each file (optional)
  echo "File: $file, Rows: $row_count"
done

# Print the total number of rows
echo "Total Row Count for Main Directory: $total_rows"

File: /data/msd/main/summary/analysis.csv.gz, Rows: 1000001
File: /data/msd/main/summary/metadata.csv.gz, Rows: 1000001
Total Row Count for Main Directory: 2000002


#### 6. Counting Rows for the Tasteprofile Directory

In [8]:
# Count the rows for each of file in the triplets and save as csv
!for i in `hdfs dfs -ls -R /data/msd/tasteprofile/triplets.tsv | awk '{print $8}'`; do echo $i ; \
hdfs dfs -cat $i | zcat | wc -l; done > triplets_count.csv

In [12]:
! hdfs dfs -ls /data/msd/tasteprofile/mismatches

Found 2 items
-rwxr-xr-x   8 jsw93 supergroup      91342 2021-09-29 10:35 /data/msd/tasteprofile/mismatches/sid_matches_manually_accepted.txt
-rwxr-xr-x   8 jsw93 supergroup    2026182 2021-09-29 10:35 /data/msd/tasteprofile/mismatches/sid_mismatches.txt


In [13]:
# Count the rows for each of file in the missmatches and save as csv
!for i in `hdfs dfs -ls /data/msd/tasteprofile/mismatches | grep '.txt$' | awk '{print $8}'`; do \
  echo "$(basename $i),$(hdfs dfs -cat $i | wc -l)"; done > mismatches_count.csv

In [10]:
%%bash 
# Define the HDFS paths for the tasteprofile directories
mismatches_dir="/data/msd/tasteprofile/mismatches"
triplets_dir="/data/msd/tasteprofile/triplets.tsv"

# Initialize a variable to store the total row count
total_rows=0

# Count rows for mismatches (.txt files)
for file in $(hdfs dfs -ls $mismatches_dir | grep .txt | awk '{print $8}')
do
  # Get the number of rows in the current .txt file
  row_count=$(hdfs dfs -cat $file | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
  
  # Print the row count for each file (optional)
  echo "File: $file, Rows: $row_count"
done

# Count rows for triplets (.tsv.gz files)
for file in $(hdfs dfs -ls $triplets_dir | grep .gz | awk '{print $8}')
do
  # Get the number of rows in the current compressed .tsv.gz file
  row_count=$(hdfs dfs -cat $file | gunzip | wc -l)
  
  # Add the row count to the total
  total_rows=$((total_rows + row_count))
  
  # Print the row count for each file (optional)
  echo "File: $file, Rows: $row_count"
done

# Print the total number of rows
echo "Total Row Count for TasteProfile Directory: $total_rows"

File: /data/msd/tasteprofile/mismatches/sid_matches_manually_accepted.txt, Rows: 938
File: /data/msd/tasteprofile/mismatches/sid_mismatches.txt, Rows: 19094
File: /data/msd/tasteprofile/triplets.tsv/part-00000.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00001.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00002.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00003.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00004.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00005.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00006.tsv.gz, Rows: 6050000
File: /data/msd/tasteprofile/triplets.tsv/part-00007.tsv.gz, Rows: 6023586
Total Row Count for TasteProfile Directory: 48393618


In [ ]:
# Run this cell before closing the notebook or kill your spark application by hand using the link in the Spark UI

stop_spark()